In [34]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [35]:
df_train = pd.read_csv(
    '../Data/train_set_long.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)

In [36]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6009430 entries, 0 to 6009429
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    bool
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 189.1 MB


In [37]:
df_test = pd.read_csv(
    "../Data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [38]:
items = pd.read_csv(
    "../Data/items.csv",
).set_index("item_nbr")

In [39]:
promo_2017_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)

In [40]:
promo_2017_train.head(5)

onpromotion                                              \
date                2017-06-13 2017-06-14 2017-06-15 2017-06-16 2017-06-17   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False      False      False      False   

                                                                           \
date               2017-06-18 2017-06-19 2017-06-20 2017-06-21 2017-06-22   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                    ...                                              \
date                ... 2017-07-30 2017-07-31 2017-08-01 2017-08-02   
store_nbr item_nbr  ...                                               
1         96995     ...      False      False      False      False   
          99197     ...      False      False      False      False   
          103520    ...      False      False      False      False   
          103665    ...      False      False      False      False   
          105574    ...      False      False      False      False   

                                                                           \
date               2017-08-03 2017-08-04 2017-08-05 2017-08-06 2017-08-07   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                               
date               2017-08-08  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 57 columns]

In [41]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

In [42]:
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [43]:
promo_2017_test.head(5)

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103501         False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False

In [44]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train
#getting the promotion of test & train and add them together!

In [1]:
df_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)

NameError: name 'df_train' is not defined

In [46]:
df_train.head(5)

unit_sales                                              \
date               2017-06-13 2017-06-14 2017-06-15 2017-06-16 2017-06-17   
store_nbr item_nbr                                                          
1         96995      0.000000   0.693147   0.000000   0.000000   0.000000   
          99197      0.693147   1.386294   1.098612   0.000000   0.693147   
          103520     1.098612   1.945910   0.693147   0.693147   0.693147   
          103665     0.693147   1.791759   0.693147   1.945910   1.098612   
          105574     1.098612   1.791759   0.693147   2.197225   1.609438   

                                                                           \
date               2017-06-18 2017-06-19 2017-06-20 2017-06-21 2017-06-22   
store_nbr item_nbr                                                          
1         96995      0.000000   0.000000   0.000000   0.000000   0.000000   
          99197      0.000000   1.098612   2.639057   0.693147   0.000000   
          103520     0.000000   1.386294   0.000000   0.693147   1.386294   
          103665     0.693147   1.386294   1.098612   0.693147   1.386294   
          105574     0.000000   2.302585   1.945910   2.944439   1.609438   

                    ...                                              \
date                ... 2017-07-30 2017-07-31 2017-08-01 2017-08-02   
store_nbr item_nbr  ...                                               
1         96995     ...   0.000000   1.098612   0.000000   0.693147   
          99197     ...   0.000000   0.000000   0.000000   0.000000   
          103520    ...   0.000000   0.693147   1.386294   0.693147   
          103665    ...   1.098612   1.098612   1.609438   1.098612   
          105574    ...   0.693147   1.609438   2.197225   2.197225   

                                                                           \
date               2017-08-03 2017-08-04 2017-08-05 2017-08-06 2017-08-07   
store_nbr item_nbr                                                          
1         96995      1.098612   0.000000   0.000000   1.098612   1.098612   
          99197      0.000000   0.000000   0.693147   0.000000   1.098612   
          103520     1.098612   1.386294   0.000000   0.000000   0.000000   
          103665     1.098612   2.197225   1.386294   0.693147   1.098612   
          105574     1.945910   1.791759   2.079442   0.000000   1.791759   

                               
date               2017-08-08  
store_nbr item_nbr             
1         96995      0.000000  
          99197      0.000000  
          103520     1.386294  
          103665     0.000000  
          105574     2.079442  

[5 rows x 57 columns]

In [47]:
df_train.columns = df_train.columns.get_level_values(1)
df_train.head(5)

date                2017-06-13  2017-06-14  2017-06-15  2017-06-16  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.693147    0.000000    0.000000   
          99197       0.693147    1.386294    1.098612    0.000000   
          103520      1.098612    1.945910    0.693147    0.693147   
          103665      0.693147    1.791759    0.693147    1.945910   
          105574      1.098612    1.791759    0.693147    2.197225   

date                2017-06-17  2017-06-18  2017-06-19  2017-06-20  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.000000    1.098612    2.639057   
          103520      0.693147    0.000000    1.386294    0.000000   
          103665      1.098612    0.693147    1.386294    1.098612   
          105574      1.609438    0.000000    2.302585    1.945910   

date                2017-06-21  2017-06-22  ...  2017-07-30  2017-07-31  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    0.000000    1.098612   
          99197       0.693147    0.000000  ...    0.000000    0.000000   
          103520      0.693147    1.386294  ...    0.000000    0.693147   
          103665      0.693147    1.386294  ...    1.098612    1.098612   
          105574      2.944439    1.609438  ...    0.693147    1.609438   

date                2017-08-01  2017-08-02  2017-08-03  2017-08-04  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.693147    1.098612    0.000000   
          99197       0.000000    0.000000    0.000000    0.000000   
          103520      1.386294    0.693147    1.098612    1.386294   
          103665      1.609438    1.098612    1.098612    2.197225   
          105574      2.197225    2.197225    1.945910    1.791759   

date                2017-08-05  2017-08-06  2017-08-07  2017-08-08  
store_nbr item_nbr                                                  
1         96995       0.000000    1.098612    1.098612    0.000000  
          99197       0.693147    0.000000    1.098612    0.000000  
          103520      0.000000    0.000000    0.000000    1.386294  
          103665      1.386294    0.693147    1.098612    0.000000  
          105574      2.079442    0.000000    1.791759    2.079442  

[5 rows x 57 columns]

In [48]:
items = items.reindex(df_train.index.get_level_values(1))

In [49]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]
#return the list of date, minus the specified period and put it as a starting date
#dt - timedelta(days=minus). minusing the date

In [51]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_train, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_train, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_train, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    
    if is_train:
        y = df_train[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [56]:
print("Preparing dataset...")
t2017 = date(2017, 6, 27)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

Preparing dataset...


In [70]:
df_val = pd.read_csv(
    '../Data/train_set_short.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)

In [71]:
promo_2017_val = df_val.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_val.columns = promo_2017_val.columns.get_level_values(1)

In [72]:
promo_2017_val.head(5)

date                2017-06-01  2017-06-02  2017-06-03  2017-06-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-06-05  2017-06-06  2017-06-07  2017-06-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-06-09  2017-06-10  ...  2017-06-19  2017-06-20  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   

date                2017-06-21  2017-06-22  2017-06-23  2017-06-24  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-06-25  2017-06-26  2017-06-27  2017-06-28  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False  

[5 rows x 28 columns]

In [73]:
df_val = df_val.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_val.columns = df_val.columns.get_level_values(1)

In [74]:
df_val.head(5)

date                2017-06-01  2017-06-02  2017-06-03  2017-06-04  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       1.386294    1.098612    1.945910    1.098612   
          103520      1.098612    1.098612    0.693147    0.000000   
          103665      0.000000    1.791759    1.791759    1.098612   
          105574      2.484907    1.791759    1.386294    1.386294   

date                2017-06-05  2017-06-06  2017-06-07  2017-06-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       1.098612    0.000000    0.000000    0.693147   
          103520      0.693147    1.609438    0.693147    0.693147   
          103665      1.386294    1.791759    1.386294    0.000000   
          105574      1.386294    2.079442    2.397895    1.945910   

date                2017-06-09  2017-06-10  ...  2017-06-19  2017-06-20  \
store_nbr item_nbr                          ...                           
1         96995       0.693147    0.000000  ...    0.000000    0.000000   
          99197       0.693147    1.609438  ...    1.098612    2.639057   
          103520      1.098612    1.386294  ...    1.386294    0.000000   
          103665      1.098612    1.609438  ...    1.386294    1.098612   
          105574      2.079442    2.079442  ...    2.302585    1.945910   

date                2017-06-21  2017-06-22  2017-06-23  2017-06-24  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.000000    0.000000    0.000000   
          103520      0.693147    1.386294    1.386294    0.693147   
          103665      0.693147    1.386294    0.693147    2.302585   
          105574      2.944439    1.609438    1.791759    1.098612   

date                2017-06-25  2017-06-26  2017-06-27  2017-06-28  
store_nbr item_nbr                                                  
1         96995       0.693147    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    1.386294    1.386294  
          103520      1.098612    0.000000    1.098612    1.386294  
          103665      1.098612    0.000000    0.000000    0.000000  
          105574      1.386294    2.302585    1.098612    1.791759  

[5 rows x 28 columns]

In [78]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_val, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_val, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_val, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017_val, t2017, 14, 14).sum(axis=1).values
    })
    
    for i in range(14):
        X["promo_{}".format(i)] = promo_2017_val[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    
    if is_train:
        y = df_val[
            pd.date_range(t2017, periods=14)
        ].values
        return X, y
    return X

In [79]:
X_val, y_val = prepare_dataset(date(2017, 6, 15))

In [80]:
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

KeyError: "None of [DatetimeIndex(['2017-08-13', '2017-08-14', '2017-08-15'], dtype='datetime64[ns]', freq='D')] are in the [columns]"